<a href="https://colab.research.google.com/github/JiaweiYu1/fasttext/blob/master/dbpedia_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import re
import os
import codecs

import tensorflow as tf

import time
import datetime

from sklearn.utils import shuffle
import csv
import html

from tensorflow.contrib import slim

import argparse
from datetime import datetime
import pickle
import pprint
import easydict
import random


#tf.enable_eager_execution()

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir("/content/drive/My Drive/Colab_Notebooks/fasttext")
cwd = os.getcwd()
print(cwd)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Colab_Notebooks/fasttext


In [0]:
class fastTextModel():
    """
    A simple implementation of fasttext for text classification
    """
    def __init__(self, sequence_length, num_classes, vocab_size,
                 embedding_size, learning_rate, l2_reg_lambda, epoch, 
                 decay_steps, decay_rate, is_training=True,
                 initializer=tf.random_normal_initializer(stddev=0.1)):
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.num_classes = num_classes
        self.sequence_length = sequence_length
        self.learning_rate = learning_rate
        self.decay_steps = decay_steps
        self.decay_rate = decay_rate
        self.epoch = epoch
        self.is_training = is_training
        self.l2_reg_lambda = l2_reg_lambda
        self.initializer = initializer
 
        self.input_x = tf.placeholder(tf.int32, [None, self.sequence_length], name='input_x')
        self.input_y = tf.placeholder(tf.int32, [None, self.num_classes], name='input_y')
 
        self.global_step = tf.Variable(0, trainable=False, name='global_step')
        self.instantiate_weight()
        self.logits = self.inference()
        self.loss_val = self.loss()
        self.train_op = self.train()
 
        self.predictions = tf.argmax(self.logits, axis=1, name='predictions')
        correct_prediction = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'), name='accuracy')
 
    def instantiate_weight(self):
        with tf.variable_scope('weights', reuse=tf.AUTO_REUSE):
            self.Embedding = tf.get_variable('Embedding', shape=[self.vocab_size, self.embedding_size],
                                             initializer=self.initializer)
            self.W_projection = tf.get_variable('W_projection', shape=[self.embedding_size, self.num_classes],
                                                initializer=self.initializer)
            self.b_projection = tf.get_variable('b_projection', shape=[self.num_classes])
 
 
    def inference(self):
        with tf.name_scope('embedding'):
            words_embedding = tf.nn.embedding_lookup(self.Embedding, self.input_x)
            self.average_embedding = tf.reduce_mean(words_embedding, axis=1)
 
        logits = tf.matmul(self.average_embedding, self.W_projection) +self.b_projection
 
        return logits
 
 
    def loss(self):
        # loss
        with tf.name_scope('loss'):
            losses = tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.input_y, logits=self.logits)
            data_loss = tf.reduce_mean(losses)
            #l2_loss = tf.add_n([tf.nn.l2_loss(cand_var) for cand_var in tf.trainable_variables()
            #                    if 'bias' not in cand_var.name]) * self.l2_reg_lambda
            #data_loss += l2_loss * self.l2_reg_lambda
            return data_loss
 
    def train(self):
        with tf.name_scope('train'):
            learning_rate = tf.train.exponential_decay(self.learning_rate, self.global_step,
                                                       self.decay_steps, self.decay_rate,
                                                       staircase=True)
 
            train_op = tf.contrib.layers.optimize_loss(self.loss_val, global_step=self.global_step,
                                                      learning_rate=learning_rate, optimizer='Adam')
 
        return train_op

In [4]:

class_size=14
learning_rate=0.1
batch_size=32
decay_steps=1000
decay_rate=0.9
sequence_length=467
vocab_size = 7191861
embedding_dim = 10
is_training=True
dropout_keep_prob=1
epoch = 10
batch_size = 4096
l2_reg_lambda = 0.01


fasttext = fastTextModel(sequence_length,
                      class_size,
                      vocab_size,
                      embedding_dim,
                      learning_rate,
                      l2_reg_lambda,
                      epoch,
                      decay_steps,
                      decay_rate,
                      is_training=True,
                    )

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
def batch_iter(x, y, batch_size=1, shuffle=True):
  data_x = np.array(x)
  data_y = np.array(y)
  data_len = len(x)
  num_batch = int((data_len-1)/batch_size)+1
    
  if shuffle:
    shuffle_indices = np.random.permutation(np.arange(data_len))
    x_shuffle = data_x[shuffle_indices]
    y_shuffle= data_y[shuffle_indices]
    
  else:
    x_shuffle=x
    y_shuffle = y
  for i in range(num_batch):
    start_index = i*batch_size
    end_index = min((i+1)*batch_size, data_len)
    yield (x_shuffle[start_index:end_index], y_shuffle[start_index:end_index])
    
    
    
def fit(train_x, train_y, x_dev, y_dev, batch_size, epoch):
  if not os.path.exists('dbpedia_csv/saves/fasttext'): os.makedirs('dbpedia_csv/saves/fasttext')
  if not os.path.exists('dbpedia_csv/train_logs/fasttext'): os.makedirs('dbpedia_csv/train_logs/fasttext')

  train_steps = 0
  best_val_acc = 0
    
  tf.summary.scalar('val_loss', fasttext.loss_val)
  tf.summary.scalar('val_accuracy', fasttext.accuracy)
  merged = tf.summary.merge_all()
    
  sess = tf.Session()
    
  writer = tf.summary.FileWriter('dbpedia_csv/train_logs/fasttext', sess.graph)
  saver = tf.train.Saver(max_to_keep=10)
  sess.run(tf.global_variables_initializer())
    
  for i in range(epoch):
    start_time = time.time()
    batch_train = batch_iter(train_x, train_y, batch_size)
    for batch_x, batch_y in batch_train:
      train_steps +=1
      feed_dict = {fasttext.input_x:batch_x, fasttext.input_y:batch_y}
      _, train_loss, train_acc = sess.run([fasttext.train_op, fasttext.loss_val, fasttext.accuracy], feed_dict = feed_dict)
        
      if train_steps % 100 ==0:
        feed_dict = {fasttext.input_x:x_dev, fasttext.input_y:y_dev}
        val_loss, val_acc = sess.run([fasttext.loss_val, fasttext.accuracy],feed_dict=feed_dict)
        
        summary = sess.run(merged, feed_dict=feed_dict)
        writer.add_summary(summary, global_step=train_steps)
        
        if val_acc >= best_val_acc:
          best_val_acc = val_acc
          saver.save(sess, "dbpedia_csv/saves/fasttext/model.ckpt", global_step=train_steps)
        msg = 'epoch:%d/%d, train_steps:%d, train_loss:%.4f, trina_acc:%.4f, val_loss:%.4f, val_acc:%.4f'
        print(msg%(i, epoch, train_steps, train_loss, train_acc, val_loss, val_acc))
    print("--- %s seconds per epoch ---" % (time.time() - start_time))   

      
def predict(x):
  sess = tf.Session()
    
  sess.run(tf.global_variables_initializer())
  saver = tf.train.Saver(tf.global_variables())
  ckpt = tf.train.get_checkpoint_state('dbpedia_csv/saves/fasttext/')
  saver.restore(sess, ckpt.model_checkpoint_path)
  #saver.restore(sess, 'saves/fasttext/checkpoint')
    
  feed_dict={fasttext.input_x:x}
  logits = sess.run(fasttext.logits, feed_dict=feed_dict)
  y_pred = np.argmax(logits, 1)
    
  return y_pred

In [0]:
file_to_read = open('data/dbpedia_train_x_1.pkl', 'rb')

dbpedia_train_x = pickle.load(file_to_read)

file_to_read.close()

In [0]:
file_to_read = open('data/dbpedia_y_train_1.pkl', 'rb')

dbpedia_y_train = pickle.load(file_to_read)

file_to_read.close()

In [0]:
file_to_read = open('data/dbpedia_test_x.pkl', 'rb')

dbpedia_x_test = pickle.load(file_to_read)

file_to_read.close()

In [0]:
file_to_read = open('data/dbpedia_test_y.pkl', 'rb')

dbpedia_y_test = pickle.load(file_to_read)

file_to_read.close()

In [0]:
dev_sample_index = -1 * int(0.1 * float(len(dbpedia_y_train)))
x_train_dbpedia, x_dev_dbpedia = dbpedia_train_x[:dev_sample_index], dbpedia_train_x[dev_sample_index:]
dbpedia_y_train_1, dbpedia_y_dev_1 = dbpedia_y_train[:dev_sample_index], dbpedia_y_train[dev_sample_index:]

In [0]:
dbpedia_y_dev_2 = np.array(dbpedia_y_dev_1).astype(np.int32).tolist()
dbpedia_y_train_2 = np.array(dbpedia_y_train_1).astype(np.int32).tolist()

In [12]:
len(dbpedia_y_train_1)

491400

In [13]:
fit(x_train_dbpedia, dbpedia_y_train_2, x_dev_dbpedia, dbpedia_y_dev_2,batch_size, epoch)

epoch:0/10, train_steps:100, train_loss:0.0899, trina_acc:0.9810, val_loss:0.0924, val_acc:0.9812
--- 118.15000224113464 seconds per epoch ---
epoch:1/10, train_steps:200, train_loss:0.0247, trina_acc:0.9946, val_loss:0.0646, val_acc:0.9858
--- 118.39497113227844 seconds per epoch ---
epoch:2/10, train_steps:300, train_loss:0.0041, trina_acc:1.0000, val_loss:0.0585, val_acc:0.9860
--- 120.29153656959534 seconds per epoch ---
epoch:3/10, train_steps:400, train_loss:0.0029, trina_acc:0.9998, val_loss:0.0567, val_acc:0.9863
--- 114.25055456161499 seconds per epoch ---
epoch:4/10, train_steps:500, train_loss:0.0013, trina_acc:1.0000, val_loss:0.0562, val_acc:0.9866
epoch:4/10, train_steps:600, train_loss:0.0011, trina_acc:1.0000, val_loss:0.0562, val_acc:0.9865
--- 119.35283946990967 seconds per epoch ---
epoch:5/10, train_steps:700, train_loss:0.0009, trina_acc:1.0000, val_loss:0.0564, val_acc:0.9866
--- 112.45448613166809 seconds per epoch ---
epoch:6/10, train_steps:800, train_loss:0.00

In [0]:
file_to_read = open('data/dbpedia_test_x.pkl', 'rb')

test_x = pickle.load(file_to_read)

file_to_read.close()

In [0]:
file_to_read = open('data/dbpedia_test_y.pkl', 'rb')

test_y = pickle.load(file_to_read)

file_to_read.close()

In [16]:
y_predict = predict(test_x)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from dbpedia_csv/saves/fasttext/model.ckpt-1200


In [0]:
c_prediction = tf.equal(y_predict, test_y)

In [18]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_y, y_predict)

array([[4822,   28,    9,    3,   11,   20,   50,    2,    0,    2,    1,
          11,    5,   36],
       [  38, 4926,    1,    1,    6,    1,   25,    1,    0,    0,    0,
           0,    0,    1],
       [  15,    3, 4880,   10,   56,    0,    3,    0,    1,    0,    0,
          13,    4,   15],
       [   1,    1,   30, 4952,   11,    0,    1,    0,    0,    2,    0,
           1,    1,    0],
       [   6,    3,   64,   10, 4906,    3,    3,    0,    0,    0,    1,
           1,    0,    3],
       [  15,    1,    0,    0,    1, 4973,    5,    0,    1,    2,    0,
           0,    2,    0],
       [  51,   28,    1,    0,    6,    8, 4877,   16,    7,    1,    0,
           2,    0,    3],
       [   0,    3,    0,    0,    0,    1,   16, 4974,    6,    0,    0,
           0,    0,    0],
       [   0,    0,    1,    0,    0,    0,    6,   11, 4981,    1,    0,
           0,    0,    0],
       [   1,    0,    0,    2,    0,    1,    0,    1,    0, 4976,   19,
           0,    

In [19]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_y, y_predict)
print('The test accuracy is', accuracy)

The test accuracy is 0.9869571428571429
